### Импорты, параметры ColBERT

In [1]:
from interface import prepare_tsv, save_index, top_n_similar_by_index, Collection, pair_scores, load_model, get_query_emb_batch, cosine_similarity_batch
import pandas as pd
import numpy as np
import os

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
experiment = "colbert-5387"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию.  

Поставить свои пути

In [2]:
pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
dst_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
src_fld = "/home/sondors/Documents/price/ColBERT/tutorial"

### Подготовка моделей

В данном примере готовим модели в качестве того на что матчить оффера. Данные должны быть в формате, пригодном для индексации с помощью ColBERT. В общем случае в индексе может быть и смесь моделей и уже ранее моделизованных офферов

In [3]:
df_models = pd.read_csv(pth_models, sep=";")
df_models

,model_id,name,brand_name,full_name,category_name,category_id
0,623742,920-005619,Logitech,Logitech 920-005619,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,Zipper Bag,Hama,Hama Zipper Bag,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,CC-3064,Nokia,Nokia CC-3064,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,CKS-X7/R,Sony,Sony CKS-X7/R,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,EP-031023,Era Pro,Era Pro EP-031023,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...
99610,7049424,MD-108,Mivo,Mivo MD-108,портативная акустика,3904
99611,7049425,MD-165,Mivo,Mivo MD-165,портативная акустика,3904
99612,7049426,Boost 20W,Rocket,Rocket Boost 20W,портативная акустика,3904
99613,7049427,Motion 10W,Rocket,Rocket Motion 10W,портативная акустика,3904


# ColBERT стандартное использование

In [3]:
id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

for cat_id in id_category.keys():
    category_models = df_models[df_models.category_id == cat_id].reset_index(drop=True)
    prepare_tsv(category_models, dst_fld, cat_id)

### Индексируем модели и сохраняем индекс на диск

В id_category всего 13 категорий, в сумме 41037 шт моделей:

- ColBERT bert-base-multilingual-cased_dim_768: индексация за 760 сек на CPU с 16 ядрами без GPU, вес индекса 109 Мб

- ColBERTv2.0_dim_128: индексация за 790 сек на CPU с 16 ядрами без GPU, вес индекса 20.6 Мб

In [4]:
for cat_id in id_category.keys():
    models_colbert = Collection(path=os.path.join(dst_fld, "tsv", f"{cat_id}_models.tsv"))
    index_name = f'models_{cat_id}_{nbits}bits'
    indexer = save_index(ckpt_pth, doc_maxlen, nbits, kmeans_niters, nranks, dst_fld, experiment, models_colbert, index_name)

[Apr 11, 12:41:39] #> Loading collection...
0M 


[Apr 11, 12:41:39] #> Note: Output directory /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_3902_2bits already exists


[Apr 11, 12:41:39] #> Will delete 10 files already at /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_3902_2bits in 20 seconds...
#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 0.0001,
    "maxsteps": 5387,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation

WARNING clustering 4558 points to 1024 centroids: please provide at least 39936 training points


[Apr 11, 12:42:03] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.022, 0.023, 0.02, 0.021, 0.021, 0.021, 0.023, 0.021, 0.017, 0.019, 0.019, 0.023, 0.02, 0.021, 0.02, 0.026, 0.02, 0.021, 0.022, 0.02, 0.021, 0.026, 0.022, 0.018, 0.02, 0.021, 0.023, 0.023, 0.024, 0.019, 0.02, 0.019, 0.018, 0.018, 0.024, 0.022, 0.019, 0.022, 0.022, 0.022, 0.02, 0.022, 0.023, 0.017, 0.021, 0.019, 0.018, 0.019, 0.019, 0.021, 0.025, 0.021, 0.024, 0.019, 0.022, 0.017, 0.018, 0.021, 0.023, 0.02, 0.019, 0.021, 0.024, 0.023, 0.019, 0.022, 0.019, 0.02, 0.017, 0.022, 0.02, 0.02, 0.022, 0.021, 0.02, 0.021, 0.019, 0.021, 0.022, 0.021, 0.021, 0.019, 0.022, 0.021, 0.021, 0.022, 0.022, 0.023, 0.023, 0.019, 0.019, 0.025, 0.023, 0.019, 0.021, 0.022, 0.019, 0.021, 0.022, 0.019, 0.017, 0.018, 0.02, 0.02, 0.023, 0.023, 0.022, 0.02, 0.021, 0.021, 0.021, 0.025, 0.019, 0.019, 0.022, 0.018, 0.02, 0.022, 0.017, 0.02, 0.025, 0.021, 0.02, 0.026, 0.02, 0.019, 0.023, 0.021]
[Apr 11,

0it [00:00, ?it/s]

[Apr 11, 12:42:03] [0] 		 #> Saving chunk 0: 	 488 passages and 4,797 embeddings. From #0 onward.
[Apr 11, 12:42:03] [0] 		 #> Checking all files were saved...
[Apr 11, 12:42:03] [0] 		 Found all files!
[Apr 11, 12:42:03] [0] 		 #> Building IVF...
[Apr 11, 12:42:03] [0] 		 #> Loading codes...
[Apr 11, 12:42:03] [0] 		 Sorting codes...
[Apr 11, 12:42:03] [0] 		 Getting unique codes...
[Apr 11, 12:42:03] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 11, 12:42:03] #> Building the emb2pid mapping..
[Apr 11, 12:42:03] len(emb2pid) = 4797
[Apr 11, 12:42:03] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_3902_2bits/ivf.pid.pt
[Apr 11, 12:42:03] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_3902_2bits/metadata.json ..


...

[Apr 11, 12:47:14] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.031, 0.031, 0.029, 0.032, 0.034, 0.031, 0.03, 0.033, 0.028, 0.032, 0.031, 0.035, 0.032, 0.035, 0.031, 0.036, 0.03, 0.031, 0.028, 0.03, 0.031, 0.034, 0.032, 0.03, 0.03, 0.035, 0.034, 0.037, 0.033, 0.032, 0.03, 0.031, 0.032, 0.031, 0.034, 0.036, 0.029, 0.03, 0.037, 0.029, 0.03, 0.033, 0.036, 0.031, 0.034, 0.032, 0.032, 0.029, 0.032, 0.03, 0.032, 0.031, 0.034, 0.03, 0.035, 0.029, 0.034, 0.029, 0.034, 0.034, 0.03, 0.03, 0.035, 0.038, 0.032, 0.031, 0.029, 0.029, 0.029, 0.03, 0.03, 0.033, 0.034, 0.031, 0.029, 0.03, 0.033, 0.031, 0.035, 0.031, 0.031, 0.029, 0.035, 0.03, 0.03, 0.03, 0.031, 0.033, 0.034, 0.029, 0.034, 0.035, 0.033, 0.028, 0.03, 0.031, 0.029, 0.034, 0.035, 0.028, 0.03, 0.028, 0.031, 0.031, 0.032, 0.032, 0.034, 0.032, 0.033, 0.032, 0.029, 0.032, 0.03, 0.028, 0.03, 0.027, 0.029, 0.031, 0.029, 0.031, 0.036, 0.03, 0.033, 0.035, 0.03, 0.031, 0.034, 0.033]
[Apr 11, 12:

0it [00:00, ?it/s]

[Apr 11, 12:47:15] [0] 		 #> Saving chunk 0: 	 1,709 passages and 15,741 embeddings. From #0 onward.
[Apr 11, 12:47:15] [0] 		 #> Checking all files were saved...
[Apr 11, 12:47:15] [0] 		 Found all files!
[Apr 11, 12:47:15] [0] 		 #> Building IVF...
[Apr 11, 12:47:15] [0] 		 #> Loading codes...
[Apr 11, 12:47:15] [0] 		 Sorting codes...
[Apr 11, 12:47:15] [0] 		 Getting unique codes...
[Apr 11, 12:47:15] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 11, 12:47:15] #> Building the emb2pid mapping..
[Apr 11, 12:47:15] len(emb2pid) = 15741
[Apr 11, 12:47:15] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_280801_2bits/ivf.pid.pt
[Apr 11, 12:47:15] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_280801_2bits/metadata.json ..


1it [00:00,  1.56it/s]
100%|██████████| 1024/1024 [00:00<00:00, 169426.72it/s]


#> Joined...


### Поиск матча по индексу

Запуск 77к офферов на поиск матча среди 13 категорий или 41к шт моделей занимает 20 мин на CPU

In [4]:
offers = {
    0: 'Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
    1: 'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
    2: 'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)'
    }

cat_id = 510401 # планшеты
index_name = f'models_{cat_id}_{nbits}bits'
models_id_colbert = Collection(path=os.path.join(src_fld, "tsv", f"{cat_id}_models_id.tsv"))

top_n = top_n_similar_by_index(offers, src_fld, nranks, experiment, index_name, models_id_colbert, 5)

[Apr 11, 17:59:30] #> Loading collection...
0M 
[Apr 11, 17:59:31] #> Loading codec...
[Apr 11, 17:59:31] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Apr 11, 17:59:32] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Apr 11, 17:59:32] #> Loading IVF...
[Apr 11, 17:59:32] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1171.27it/s]

[Apr 11, 17:59:32] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 344.36it/s]


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1, 19102,  1194, 29436, 28995, 29753, 15290, 22919, 19102,
         9088, 21628,  1055,  2620,  1010,  1022,  1183, 29740,  1013, 11899,
         1183, 29740,  1010, 15536,  1011, 10882,  1009, 12562,  1010,  1196,
        14150,   102])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])




3it [00:00, 130.12it/s]


### Получаем выдачу топ N моделей для каждого оффера

In [5]:
id_to_name = dict(zip(df_models['model_id'], df_models['full_name']))
for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['similarity'][j]
        model = id_to_name[id]
        print(f"\t{id}: {model} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 25.52
	5144479: Samsung Galaxy Tab S8 Ultra --> 22.27
	3843929: Samsung Galaxy Tab S4 10.5 SM-T835 --> 21.06
	623215: Samsung Galaxy Tab S 8.4 SM-T700 --> 19.8
	4892891: Samsung Galaxy Tab A8 10.5 128Gb --> 19.64
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 25.11
	5144479: Samsung Galaxy Tab S8 Ultra --> 21.67
	3843929: Samsung Galaxy Tab S4 10.5 SM-T835 --> 21.3
	1464907: Samsung Galaxy Tab S3 9.7 SM-T820 --> 19.66
	623215: Samsung Galaxy Tab S 8.4 SM-T700 --> 19.5
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 23.27
	4509798: Samsung Galaxy Tab S7+ 12.4 128Gb --> 19.52
	5144478: Samsung Galaxy Tab S8 --> 19.48
	623213: Samsung Galax

In [6]:
top_n

[{'model_ids': [5144478, 5144479, 3843929, 623215, 4892891],
  'similarity': [25.515625, 22.265625, 21.0625, 19.796875, 19.640625]},
 {'model_ids': [5144478, 5144479, 3843929, 1464907, 623215],
  'similarity': [25.109375, 21.671875, 21.296875, 19.65625, 19.5]},
 {'model_ids': [5144477, 4509798, 5144478, 623213, 5144479],
  'similarity': [23.265625, 19.515625, 19.484375, 19.484375, 18.1875]}]

# ColBERT нестандартное использование

### Получение similarity без индексирования

Так же можно использовать ColBERT без предварительного индексирования и поиска матча по индексу. Возможно получение similarity для пары строк, для одной строки со списком строк или для каждой пары элементов из двух списков строк. 

In [7]:
query1 = "Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A)"
document1 = "Смартфон Apple iPhone 12 Pro Max 256Gb Gold"
document2 = "Apple iPhone Pro Max 11"
document3 = "модель iPhone номер 11 серия Pro Max бренд Apple"

scores = pair_scores(ckpt_pth, doc_maxlen, nbits, kmeans_niters, [query1], [document1, document2, document3], 100, "cpu")
print(f"{query1}:")
print(f"similarity = {scores[0][0]}: {document1}")
print(f"similarity = {scores[0][1]}: {document2}")
print(f"similarity = {scores[0][2]}: {document3}")


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1196, 29745, 10260, 16856, 22919, 29749, 14150, 18947,
         6207, 18059,  2340,  4013,  4098, 17273, 18259,  2751,  1006, 11460,
         3207,  2509,  6820,  1013,  1037,  1007,   102,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0])

batch: 1/1
Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A):
similarity = 23.390625: Смартфон Apple iPhone 12 Pro Max 256Gb Gold
similarity = 27.34375: Apple iPhone Pro Max 11
similarity = 23.71875: модель iPhone номер 11 серия Pro Max бренд Apple
